In [94]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import random

In [3]:
pd.options.display.max_columns = None

In [4]:
ox.settings.timeout = 360
ox.settings.timeout

360

In [5]:
tags = {'building': True}

In [6]:
gdf = ox.geometries_from_place('Wrocław', tags)

In [12]:
for column in gdf.columns:
    print(column)

unique_id
osmid
element_type
entrance
geometry
amenity
barrier
artwork_type
brand:wikidata
brand:wikipedia
location
name
name:en
network
tourism
access
operator
wheelchair
opening_hours
artist_name
image
start_date
addr:city
addr:housenumber
addr:street
addr:postcode
shop
parking
foot
addr:street:sym_ul
WroclawGIS:addr:date
WroclawGIS:addr:id
WroclawGIS:addr:layer
WroclawGIS:addr:postcode:id
WroclawGIS:addr:postcode:layer
addr:country
source:addr
man_made
building:colour
building:levels
roof:colour
roof:shape
inscription
note
brand
description
level
roof:orientation
heritage
heritage:operator
historic
ref:nid
building
fuel:GTL_diesel
fuel:HGV_diesel
fuel:diesel
fuel:discount
fuel:lpg
fuel:octane_95
fuel:octane_98
payment:american_express
payment:bancomat
payment:coins
payment:maestro
payment:mastercard
payment:notes
payment:visa
payment:visa_debit
payment:visa_electron
phone
toilets
website
check_date
highway
covered
lit
surface
height
ref
fixme
layer
fee
source
traffic_calming
emergen

In [43]:
gdf['geometry'][10].centroid.geom_type

'Point'

In [21]:
wroc_map = gpd.read_file('../ECC/EtapII-REJONY_wroclaw.shp')
city_regions = wroc_map[['NUMBER', 'geometry']]
city_regions.columns = ['REGION', 'geometry']

In [22]:
city_regions = city_regions.to_crs(epsg=4326)

In [23]:
city_regions

,REGION,geometry
0,22,"POLYGON ((17.03285 51.09598, 17.03287 51.09550..."
1,23,"POLYGON ((17.02933 51.09791, 17.02855 51.09677..."
2,24,"POLYGON ((17.02483 51.09980, 17.02329 51.09859..."
3,25,"POLYGON ((17.04817 51.11196, 17.04648 51.11242..."
4,26,"POLYGON ((17.05452 51.11637, 17.05571 51.11628..."
...,...,...
370,404,"POLYGON ((17.05453 51.09557, 17.05465 51.09548..."
371,451,"POLYGON ((17.08507 51.10518, 17.08556 51.10512..."
372,452,"POLYGON ((17.09739 51.10281, 17.09742 51.10288..."
373,453,"POLYGON ((17.02857 51.09104, 17.02871 51.09101..."


In [33]:
buildngs_data = pd.DataFrame(columns=["ID", "Type", "Geometry", "Region"])
buildngs_data["ID"] = gdf["osmid"]
buildngs_data["Type"] = gdf["building"]
buildngs_data["Geometry"] = gdf["geometry"]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
buildngs_data

,ID,Type,Geometry,Region
0,1254616835,yes,POINT (16.88293 51.14432),NaN
1,5663863964,yes,POINT (17.06296 51.12232),NaN
2,5996481764,commercial,POINT (17.03081 51.09847),NaN
3,6754512655,yes,POINT (16.92082 51.15736),NaN
4,7555071783,yes,POINT (16.89670 51.17148),NaN
...,...,...,...,...
155872,11822636,commercial,"POLYGON ((17.01321 51.11389, 17.01318 51.11377...",NaN
155873,11897959,commercial,"POLYGON ((17.01447 51.11327, 17.01449 51.11329...",NaN
155874,12035724,yes,"POLYGON ((17.03347 51.08042, 17.03310 51.08045...",NaN
155875,12102932,yes,"POLYGON ((17.01402 51.08758, 17.01403 51.08758...",NaN


In [44]:
def point_in(regions, point):
    for index, row in regions.iterrows():
        if point.within(row['geometry']):
            return row['REGION']
    return 0  

In [54]:
def map_building_to_region(buildings, regions):
    for index, row in tqdm(buildings.iterrows(), total=buildings.shape[0]):
        region = 0
        if row['Geometry'].geom_type == 'Point':
            region = point_in(regions, row['Geometry'])
        elif row['Geometry'].geom_type == 'Polygon':
            region = point_in(regions, row['Geometry'].centroid)
        buildings.at[index,'Region'] = region
    return buildings

In [58]:
buildings = map_building_to_region(buildngs_data, city_regions)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████████████████████████████████████████████████████████████████████| 155877/155877 [59:17<00:00, 43.82it/s]


In [62]:
buildings

,ID,Type,Geometry,Region,amenity
0,1254616835,yes,POINT (16.88293 51.14432),231,fuel
1,5663863964,yes,POINT (17.06296 51.12232),369,recycling
2,5996481764,commercial,POINT (17.03081 51.09847),339,NaN
3,6754512655,yes,POINT (16.92082 51.15736),204,fuel
4,7555071783,yes,POINT (16.89670 51.17148),222,NaN
...,...,...,...,...,...
155872,11822636,commercial,"POLYGON ((17.01321 51.11389, 17.01318 51.11377...",48,NaN
155873,11897959,commercial,"POLYGON ((17.01447 51.11327, 17.01449 51.11329...",48,NaN
155874,12035724,yes,"POLYGON ((17.03347 51.08042, 17.03310 51.08045...",140,NaN
155875,12102932,yes,"POLYGON ((17.01402 51.08758, 17.01403 51.08758...",59,NaN


In [61]:
buildings['amenity'] = gdf['amenity']

In [63]:
buildings.to_csv("building.csv")

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
buildings.Type.unique()

array(['yes', 'commercial', 'entrance', 'semidetached_house',
       'residential', 'apartments', 'garage', 'house', 'roof', 'detached',
       'greenhouse', 'stable', 'hotel', 'collapsed', 'office',
       'farm_auxiliary', 'retail', 'garages', 'warehouse', 'church',
       'school', 'carport', 'kindergarten', 'grandstand', 'shed',
       'service', 'transformer_tower', 'terrace', 'construction',
       'government', 'chapel', 'water_tower', 'industrial', 'riding_hall',
       'pedestal', 'train_station', 'waste', 'ruins', 'hospital',
       'university', 'civic', 'kiosk', 'gymnasium', 'sports_hall',
       'parking', 'supermarket', 'guardhouse', 'allotment_house',
       'bunker', 'dormitory', 'public', 'guard_shed', 'convent',
       'presbytery', 'hut', 'synagogue', 'container', 'college', 'semi',
       'barn', 'manufacture', 'toilets', 'steps', 'no', 'ramp',
       'cathedral', 'gate', 'fire_station', 'religious', 'stairs',
       'porch', 'hostel', 'castle', 'hangar', 'houseboat

In [74]:
building_dict = {'szkola'  : ['school'], 
                 'praca'   : ['office'],
                 'inne'    : ['commercial', 'hotel', 'retail', 'warehouse', 'church', 'kindergarten', 'service', 'civic', 
                              'kiosk', 'gymnasium', 'sports_hall', 'supermarket', 'allotment_house', 'synagogue', 'cathedral',
                              'religious', 'hostel', 'concert_hall','swimming_pool', 'stadium', 'hospital'], 
                'uczelnia' : ['university', 'college'],
                'dom'      : ['yes', 'semidetached_house', 'residential', 'apartments', 'house', 'dormitory']}

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
def sample_building(region, trip_type, buildings_df, building_dict):
    buildings_df = buildings_df[buildings_df['Region']==region]
    buildings_df = buildings_df[buildings_df['Type'].isin(building_dict[trip_type])]
    print(buildings_df.shape)
    return buildings_df['Geometry'].sample(n=1, random_state=random.randint(0,500))

In [113]:
print(sample_building(150, 'inne', buildings, building_dict))

(7, 5)
97228    POLYGON ((16.96318 51.14020, 16.96342 51.14020...
Name: Geometry, dtype: geometry
